# Gurobi optimization using multiple machine learning models
## Optimize for Price and Supply of Avocados - time horizon

THIS IS THE SAME OPTIMIZATION PROBLEM OF NOTEBOOK 2 BUT WITH THE DIFFERENCE TO CONSIDERING A TIME HORIZON PLANNING. SO OPTIMIZATION PROBLEM IS RE DEFINED TO ADD TIME BUT THE MACHINE LEARNINGS MODELS OF NOTEBOOKS 2 ARE THE SAME.
**SO, The ml models predict to one time horizon (and it is the same of notebook 2), but the optimization model consider the time horizon**

**DOCUMENTATION**
- In this example there multiple linear regressions, but gurobi machine learning acept multiple models. Documentation **"gurobi-machinelearning"**

https://gurobi-machinelearning.readthedocs.io/en/stable/api.html


- In addition, to define the decision variables, parameters, restriction, etc of the optimization model are used **"gurobipy-pandas"**. Using this package is possible define the optimization model using pandas DataFrames

https://gurobipy-pandas.readthedocs.io/en/stable/

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

### 0. Root repo

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

## PREPARATION

### 1. Load data needs to use
In this example data is loaded because it is necesary to generate parameters of optimization model

IMPORTANT: the data is defined to each region but in one unique time interval, but now the optimization problem is defined to multi time optimization, so for this example, it assume that the data is the same in all times

In [ ]:
# read data that have all the units sold for each region
path_data_basic_features = 'artifacts/data/data_basic_features.pkl'
data_units_sold = pd.read_pickle(path_data_basic_features)

In [ ]:
##### use data to generate parameters for optimization model

# min, max deliry each region
data_min_delivery_mean = data_units_sold.groupby("region")["units_sold"].min().rename('min_delivery')
data_max_delivery_mean = data_units_sold.groupby("region")["units_sold"].max().rename('max_delivery')

# historical distribution of price each region
data_historical_max_price = data_units_sold.groupby("region")["price"].max().rename('max_price')


list_regions = ['Great_Lakes', 'Midsouth', 'Northeast'] # TODO: LIMIT SIZE TO GUROBI FREE LICENCE
data_min_delivery_mean = data_min_delivery_mean[list_regions]
data_max_delivery_mean = data_max_delivery_mean[list_regions]
data_historical_max_price = data_historical_max_price[list_regions]

In [ ]:
data_min_delivery_mean[list_regions]

In [ ]:
data_historical_max_price

### 2. Load model machine learning
Load models that given an input (price of each regions and other features) predict the price (One different model to predict the price of each region)

The model was trained in the notebook "models/5_prices_regions_multiple_lr"

In [ ]:
# params

# path folder models
path_folder_artifacts = 'artifacts/models/5_prices_regions_multiple_lr-gurobi-free-licence/'

# list models names "model_name".pkl
list_models_names = ['Great_Lakes', 'Midsouth', 'Northeast'] # TODO: LIMIT SIZE TO GUROBI FREE LICENCE

In [ ]:
### load models
dict_models = {}
for model_name in list_models_names:
    print(f'loading model: {model_name}')
    path_model = path_folder_artifacts + f'model_{model_name}.pkl'
    with open(path_model, 'rb') as artifact:
        dict_models[model_name] = pickle.load(artifact)

## RUN OPTIMIZATION

### 0. Load transversal params - sets of optimization model
Transversal all codes, not only this code. For example order in features in the data.

Save the sets of optimization model as pandas index

#### Define sets 
Ouput pandas index with the values of each set

In [ ]:
############## set regions - index pandas ##############

# generate a pandas index with the values of the regions. This works as sets of optimization model
list_regions = ['Great_Lakes', 'Midsouth', 'Northeast'] # TODO: LIMIT SIZE TO GUROBI FREE LICENCE
regions = list_regions

# index pandas region
index_regions = pd.Index(regions)
index_regions

In [ ]:
############## set time - index pandas ##############

# generate a pandas index with time horizon planning
time_horizon_planning = ['t1', 't2', 't3', 't4'] # TODO: LIMIT SIZE TO GUROBI FREE LICENCE

# index pandas time
index_time = pd.Index(time_horizon_planning)
index_time

In [ ]:
############## multi set region&time - index pandas ##############

#index pandas multiindex region-time
index_region_time = pd.MultiIndex.from_product((regions, time_horizon_planning), 
                                               names = ('region', 'time')
                                              )
index_region_time

### 1. Create guroby optimization model
Documentation: https://www.gurobi.com/documentation/current/refman/py_model.html

In [ ]:
# env = gp.Env(params=params)

#Create the model within the Gurobi environment
model_opt = gp.Model(name = "Avocado_Price_Allocation_time_horizon")

### 2. Upper bounds and lower bounds of decision variables
Values that are boundss in decision variables. In gurobi the upper and lower boundss could be defined in the same moment that variables are created and not are defined as restrictions explicitly 

- $a_{min},a_{max}$: minimum and maximum price ($\$$) per avocado (price is a input of machine learning model)
- $b^r_{min},b^r_{max}$: minimum and maximum number of avocados allocated to region $r$

In [ ]:
# a_min, a_max: min and max price of product A. 
# IMPORTANT IF A UNIQUE VALUE IS DEFINED, GUROBI ADAPT THE VALUE TO THE FORMAT OF VARIABLE (repeat value if the decision variable is defined to one set or multiple sets)
a_min = 0
a_max = 2


# IMPORTANT THE DATA IS DEFINED TO SET "regions", but the decision variable has sets "regions" and "time". REPLICATE THE VALUE FOR EACH TIME
# b_min(r), b_max(r): min and max historical products send to each region (value get from historical data)
b_min = data_min_delivery_mean
b_max = data_max_delivery_mean

# transform into bounds for decision variable with indexes 2 sets (regio  and time)
b_min_time = pd.Series(index = index_region_time, name = 'min_delivery')
for region in regions:
    for time_horizon in time_horizon_planning:
        b_min_time.loc[region] = b_min.loc[region]

b_max_time = pd.Series(index = index_region_time, name = 'max_delivery')
for region in regions:
    for time_horizon in time_horizon_planning:
        b_max_time.loc[region] = b_max.loc[region]

In [ ]:
# show lower bound - if the limitd is defined as scalar, gurobi can repeated in the decision variable for each sets
a_min

In [ ]:
# show lower bound - if the limit is defined as a vector, its dimensions needs to be the same as the dimensions of decision variable
b_min_time

In [ ]:
b_max_time

### 3. Input parameters of optimization model
##### That are not decision variables either parameters of machine learning model)

**Set**
- $r$ : will be used to denote each region


**Parameters Optimization Model**
- $B (t)$: available avocados to be distributed across the regions.Total amount of avocado supply. The amount change between periods (t)

- $c_{waste}$: cost ($\$$) per wasted avocado

- $c^r_{transport}$: cost ($\$$) of transporting a avocado to region $r$

In [ ]:
# B: supply product
B_time = pd.Series(
    {
        "t1": 30,
        "t2": 10,
        "t3": 30,
        "t4": 55
    }, name = 'supply') # # TODO: LIMIT SIZE TO GUROBI FREE LICENCE
B_time = B_time.loc[time_horizon_planning]

# c_waste: cost of waste product
c_waste = 0.1


# c_transport(r): cost transport for each region
c_transport = pd.Series(
    {
        "Great_Lakes": 0.3,
        "Midsouth": 0.1,
        "Northeast": 0.4
    }, name='transport_cost') # TODO: LIMIT SIZE TO GUROBI FREE LICENCE
c_transport = c_transport.loc[regions]

### 4. Features input machine learning model fixed (that are not decision variables or parameters in optimization model)
Define the features that are inputs of machine learning model that are not decision variables of optimization model (so this values doesn't change). And also, this features that are not parameters of optimization model, so this values are not used in the restrictions

In [ ]:
# seasonality: 1 if it is the peak season; 0 if isn't
peak_or_not = 0
peak_or_not

In [ ]:
index_time

In [ ]:
# generate a dataframe with the "fixed" features of optimization model. 
# This is an instance of machine learning model. In this part only have the features that have fixed values for this optimization
instance_ml_model = pd.DataFrame(
    data={
        "peak": peak_or_not
    },
    index=index_time
)
instance_ml_model

### 5. Decision variables of optimization model

Let us now define the decision variables. In our model, we want to store the price and number of avocados allocated to each region. We also want variables that track how many avocados are predicted to be sold and how many are predicted to be wasted. 

- $p(r, t)$ the price of an avocado ($\$$) in each region in each time. The maxium price. It is a feature of machine learning model
- $x(r, t)$ the number of avocados supplied to each region in each time
- $s(r, t)$ the predicted number of avocados sold in each region in each time
- $u(r, t)$ the predicted number of avocados unsold (wasted). (Inventory) in each region each time
- $d(r, t)$ the predicted demand in each region in each time. It is the target of machine learning model (because this value change according the input, it is a decision variable)

All those variables are created using gurobipy-pandas, with the function `gppd.add_vars`. To use this function it is necessary to define:
- model: optimization model of gurobi
- index: pandas index. With this index it can defined the sets of the decision variables
- name: name of the decision variable
- Example: x = gppd.add_vars(model, index, name="x")

In [ ]:
# p(r): price. feature of machine learning model
p = gppd.add_vars(model_opt, index_region_time, lb = a_min, ub = a_max, name = 'price') # bounds prices


# x(r): supply
x = gppd.add_vars(model_opt, index_region_time, lb = b_min_time, ub= b_max_time, name = 'supply') # bounds supply - using historical data


# s(r): solds given a certain price
s = gppd.add_vars(model_opt, index_region_time, lb = -gp.GRB.INFINITY, name = "solds")


# u(r): inventary. units not sold. waste.
u = gppd.add_vars(model_opt, index_region_time, lb = -gp.GRB.INFINITY, name = "inventory") 


# d(r): demand. output of machine learning model
d = gppd.add_vars(model_opt, index_region_time, lb = -gp.GRB.INFINITY, name = "demand_predicted") # BY DEFULT LOWER BOUND IS ZERO



### 6. Constraints (constraints that are not generated by a ml model)

#### 6.1 Add the Supply Constraint
Make sure that the total number of avocados supplied is equal to $B$
\begin{align*} \sum_{r} x_r(t) &= B(t)        \:\:\:\:\forall t \end{align*} 

In [ ]:
# supply for each time

constraint_supply = gppd.add_constrs(model_opt, x.groupby('time').sum(), gp.GRB.LESS_EQUAL, B_time, name = 'supply periods')
constraint_supply

#### 6.2 Add Constraints That Define Sales Quantity
The sales quantity is the minimum of the allocated quantity and the predicted demand, i.e., $s_r = \min \{x_r,d_r(p_r)\}$ This relationship can be modeled by the following two constraints for each region $r$.

\begin{align*} s_r(t) &\leq x_r(t)                \:\:\:\:\forall r \forall t\\
s_r(t) &\leq d(r)(t)                   \:\:\:\:\forall r \forall t\end{align*}

In [ ]:
constraint_solds_supply = gppd.add_constrs(model_opt, s, gp.GRB.LESS_EQUAL, x, name = 'solds <= supply')
constraint_solds_supply

In [ ]:
constraint_solds_demand = gppd.add_constrs(model_opt, s, gp.GRB.LESS_EQUAL, d, name = 'solds <= demand')
constraint_solds_demand

#### 6.3 Add the Wastage Constraints
Define the predicted unsold number of avocados in each region, given by the supplied quantity that is not sold. For each region $r$.

\begin{align*}
u_r(t) &= x_r(t) - s_r(t)                 \:\:\:\:\forall r \forall t
\end{align*}

In [ ]:
constraint_wastage = gppd.add_constrs(model_opt, u, gp.GRB.EQUAL, x - s, name = 'wastage')
constraint_wastage

#### 6.4 Model update - add the constraint to gurobi model

In [ ]:
model_opt.update()

In [ ]:
### show all decision variables - debugging problems - validate after compile decision varaibles
#x.gppd.VarName  # see name
#x.gppd.ub # upper bound
x.gppd.lb # lowe bound

#p.gbpd.X # see value of decision variable - only works after optimization

In [ ]:
model_opt

### 7. Add constraints that are machine learning models
To add constraints that have machine learning models it is necessary define a dataframe that are the instance of prediction (it has columns as gurobi decision variables) and then create the constraint in gurobi.

In this example, where each region has its own model, the dataframe instance also needs to be defined indidually. For the decision variable that are defined in the set "regions" it is important filter the dataframe instance with the correct element of the set region

**So, for each element in set region will be defined the instance dataframe and a constraint. Each region has it own model**Also, the instance has only one row, so now it is possible define a optimization model with set "time" and each row of the dataframe could be the instance of time t, t+1, t+2, etc


**IMPORTANT: LOGICALLY, FOR THIS EXAMPLE, TO DEFINE THE CONSTRAINTS OF ML MODELS, A FOR COULD HAVE BEEN MADE IN THE SET "REGIONS" BUT IT WAS NOT DONE CONSCIOUSLY THINKING OF AN EXAMPLE IN WHICH RESTRICTIONS HAVE TO BE DEFINED IN DIFFERENT SETS**

In [ ]:
############ create instance for predict demand fo each time ############



print('-- loading constraints machine learning models --')
for region in regions:
    print(f'\n\nloading constraints of demand of region: {region}')

    # there is a dataframe with features fixed (no decision variables). ROWS "time"
    aux_features_fixed = instance_ml_model
    
    # create a dataframe with decision variables gurobi. filter it by time. In this example the price of all regions are features of the ml model
    aux_features_decision =  pd.DataFrame(p.unstack(level = 0))
    
    #name_columns_feature_decision = aux_features_decision.columns # CORRECTION NAME COLUMNS TO BE THE SAME COLUMNS NAMES IN DATAFRAME USED TO TRAIN
    name_columns_feature_decision = ['price_' + name_region for name_region in list_regions]
    name_columns_feature_decision = [column.lower() for column in name_columns_feature_decision]
    aux_features_decision.columns = name_columns_feature_decision
    
    # join into a dataframe instance
    instance = pd.concat([aux_features_fixed, aux_features_decision], axis=1) # generate instance
    
    
    ############ create constraint based in machine learning model ############
    # load model
    model_ml = dict_models[region]
    
    ## add model to predict the demand for each region in differents time horizon with the SAME MODEL
    ml_constraint = add_predictor_constr(gp_model = model_opt, 
                                       predictor = model_ml, 
                                       input_vars = instance,  #ROWS "time"
                                       output_vars = d[region], # filter decision variable for the element of the set region, ROWS "time"
                                       name = f'model_predict_{region}'
                                      )
    ml_constraint.print_stats()

In [ ]:
# example instance
instance

#### DOCUMENTATION GUROBI MACHINE LEARNING

Call
[add_predictor_constr](https://gurobi-machinelearning.readthedocs.io/en/stable/auto_generated/gurobi_ml.add_predictor_constr.html)
to insert the constraints linking the features and the demand into the model `m`.

It is important that you keep the columns in the order above, otherwise you will see an error. The columns must be in the same order as the training data.

Obs: to add this constraints the way is little different and it is not neccesary call model.update()

**Documentation - parameters**

- gp_model (gurobipy model) – The gurobipy model where the predictor should be inserted.

- predictor – The predictor to insert.

- input_vars (mvar_array_like) – Decision variables used as input for predictor in gp_model.

- output_vars (mvar_array_like, optional) – Decision variables used as output for predictor in gp_model.

### 8. Define Objetive Function
The goal is to maximize the **net revenue**, which is the product of price and quantity, minus costs over all regions. This model assumes the purchase costs are fixed (since the amount $B$ is fixed) and are therefore not incorporated.

\begin{align} 
\textrm{maximize} &  \sum_{r}\sum_{t}  (p_r * s_r - c_{waste} * u_r -
c^r_{transport} * x_r)& 
\end{align}

In [ ]:
### sum values across the set "time". show the value for each "region"
#x.groupby('region').sum()

In [ ]:
model_opt.setObjective((p * s).sum() - c_waste * u.sum() - (c_transport * x.groupby('region').sum()).sum(),
                       gp.GRB.MAXIMIZE)

### 9. Solve optimization problem
The objective is **quadratic** since we take the product of price and the predicted sales, both of which are variables. Maximizing a quadratic
term is said to be **non-convex**, and we specify this by setting the value of the [Gurobi NonConvex
parameter](https://www.gurobi.com/documentation/10.0/refman/nonconvex.html) to be $2$.

#### 9.1 Solve optimization problem

In [ ]:
# solve cuadratic problems
model_opt.Params.NonConvex = 2

In [ ]:
model_opt

In [ ]:
# solve
model_opt.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
model_opt.Status

#### 9.2 Save optimal values in a dataframe
To get the optimal values of decision variables it is neccesary call "var.gppd.X"

In [ ]:
# create dataframe with index
solution = pd.DataFrame(index = index_region_time)

# save optimal values
solution["Price"] = p.gppd.X
#solution["Historical_Max_Price"] = data_historical_max_price  # this is informative value get from historical data
solution["Allocated"] = x.gppd.X
solution["Sold"] = s.gppd.X
solution["Wasted"] = u.gppd.X
solution["Pred_demand"] = d.gppd.X

# round values
solution = solution.round(3)


# get value objetive function
opt_revenue = model_opt.ObjVal

In [ ]:
# show value objetive function
print("\n The optimal net revenue: $%f million" % opt_revenue)

In [ ]:
# show value decision variables
solution

In [ ]:
p.gppd.X

In [ ]:
index_region_time

# debugg Model is infeasible or unbounded

In [ ]:
# model_opt.computeIIS()

## show constraint supply

In [ ]:
constraint_supply

In [ ]:
model_opt.getRow(constraint_supply['t1'])